In [ ]:
import os
import glob
import psycopg2
import pandas as pd
from sql_queries import *

In [ ]:
conn = psycopg2.connect("host=127.0.0.1 dbname=sparkifydb user=student password=student")
cur = conn.cursor()

In [ ]:
def get_files(filepath):
    all_files = []
    for root, dirs, files in os.walk(filepath):
        files = glob.glob(os.path.join(root,'*.json'))
        for f in files :
            all_files.append(os.path.abspath(f))
    
    return all_files

In [ ]:
song_files= get_files('data\song_data\')

In [ ]:
filepath = song_files[0]

In [ ]:
df=pd.read_json(filepath,lines=True)

In [ ]:
df.head()

In [ ]:
song_data=df[['song_id','title','artist_id','year','duration']].values[0].tolist()

In [ ]:
song_data

In [ ]:
cur.execute(song_table_insert, song_data)
conn.commit()

In [ ]:
artist_data = df[['artist_id','artist_name','artist_location','artist_latitude','artist_longitude']].values[0].tolist()

In [ ]:
artist_data

In [ ]:
cur.execute(artist_table_insert, artist_data)
conn.commit()

In [ ]:
log_files = get_files('data\log_data\')

In [ ]:
filepath = log_files[0]

In [ ]:
df = pd.read_json(filepath, lines=True)
df.head()

In [ ]:
df = df.loc[df['page'] == 'NextSong']
df.head()

In [ ]:
t= pd.to_datetime(df['ts'], unit='ms') 
t.head()

In [ ]:
time_data = []
for line in t:
    time_data.append([line, line.hour, line.day, line.week, line.month, line.year, line.day_name()])
column_labels = ('start_time', 'hour', 'day', 'week', 'month', 'year', 'weekday')

In [ ]:
time_df = pd.DataFrame.from_records(time_data, columns=column_labels)
time_df.head()

In [ ]:
for i, row in time_df.iterrows():
    cur.execute(time_table_insert, list(row))
    conn.commit()

In [ ]:
user_df = df[['userId','firstName','lastName','gender','level']]

In [ ]:
for i, row in user_df.iterrows():
    cur.execute(user_table_insert, row)
    conn.commit()

In [ ]:
for index, row in df.iterrows():

    # get songid and artistid from song and artist tables
    cur.execute(song_select, (row.song, row.artist, row.length))
    results = cur.fetchone()
    
    if results:
        songid, artistid = results
    else:
        songid, artistid = None, None
   

In [ ]:
# insert songplay record
songplay_data = (index, pd.to_datetime(row.ts, unit='ms'), int(row.userId), row.level, songid, artistid, row.sessionId, row.location, row.userAgent)
cur.execute(songplay_table_insert, songplay_data)
conn.commit()